# Report 3

### by Kaitlyn Keil and Kevin Zhang
### April 2017

<a href = http://tna.europarchive.org/20110116113217/http:/www.food.gov.uk/science/dietarysurveys/dietsurveys/>UK Food Network Dataset</a>

In [9]:
""" This file contains code used to analyze the
UK Food Nutrient Database, the information for which can be found
at this link: http://tna.europarchive.org/20110116113217/http:/www.food.gov.uk/science/dietarysurveys/dietsurveys/

The following packages are required to run this code.
"""

from __future__ import print_function, division
import pandas as pd

import sys
import numpy as np
import math
import thinkstats2
import thinkplot
import matplotlib.pyplot as plt
from operator import itemgetter
from cycler import cycler
from sklearn.feature_extraction import DictVectorizer
%matplotlib inline
import seaborn as sns

from collections import defaultdict, Counter
import statsmodels.formula.api as smf

In [166]:
### This is for the other massive set

def ReadIngredients(region='African'):
    """ Takes the ingedients dataset and converts it into a Dataframe. 
    Region can be: African, EastAsian, EasternEuropean, LatinAmerican,
        MiddleEastern, NorthAmerican, NorthernEuropean, SouthAsian,
        SoutheastAsian, SouthernEuropean, WesternEuropean"""
    temp = pd.read_csv("FoodDataset/ingredients.csv") # Reads the file with a lot of NaNs and little order
    #temp.fillna(value='None')
    temp2 = temp[temp.cuisine_type==region]
    temp_dict = temp2.to_dict('index')
    
    temp_list = []
    
    for keys,values in temp_dict.items():
        td = {}
        for key,x in values.items():
            if key=='cuisine_type':
                td[key] = x
            else:
                td[x]=1
        temp_list.append(td)
        
        
    return pd.DataFrame(temp_list).fillna(0)

def corrPlot(df):
    """ Given the ingredients dataframe, selects
    the most used ingredients and creates a correlation 
    matrix from them."""
    df1 = df.ix[:, (df.columns != 'cuisine_type')]
    temp_df = df1.sum()
    mdf = temp_df[temp_df > 50].iloc[1:]
    print(mdf)
    corr_df = pd.DataFrame()
    
    for x in mdf.index:
        if x!='index1':
            corr_df[x] = df[x]
        
    corr_df['index1'] = [int(x) for x in corr_df.index.values]
    sns.heatmap(corr_df.corr())
    return corr_df

#ingredients = ReadIngredients('SoutheastAsian')

# Shows what ingredients are most/least likely to be used together.
#  For example, in African cuisine, cumin and coriander are fairly 
#  correlated, whereas vegetable oil and olive oil together are not.
#%matplotlib inline
#corr_df = corrPlot(ingredients)

In [52]:
def ReadProximates():
    df = pd.read_excel('dietary.xls', sheetname='Proximates')
    column_list = ['Water (g)', 'Protein (g)', 'Fat (g)', 'Carbohydrate (g)', 'Total sugars (g)']
    for column in column_list:
        new_name = column[:-4]
        df[new_name]= df[column]
        for key, value in df[column].items():
            if isinstance(value, str):
                if '(' in value:
                    df[new_name][key] = float(value[1:-1])
                elif value=='Tr':
                    df[new_name][key] = 0.0
                elif value=='N':
                    df[new_name][key] = 0.0
                elif math.isnan(float(value)):
                    df[new_name][key] = 0.0
                    print('NaN found')
                else:
                    df[new_name][key] = float(value)
        
    return df

In [ ]:
#tester = pd.read_excel('dietary.xls', sheetname='Proximates')
tester = ReadProximates()
# TO DO
# For cleaning: 
#  Get rid of Ns and Trs <- Done!
#  Figure out how to deal with parentheses <- Done!
#  Convert rows into floats (.astype) <- Done!
#  Get ready for lots of brackets...
#  Potentially just make different columns for the integer values <- ...done?

# For report:
#  Figure out if we can actually classify based on these values
#  Make a dictionary mapping from the Group codes to actual names
#  Figure out a good model
#  Do the thing
#  PCA to figure out the components we will want to look at
#  Unclassified ML to figure out natural groups?


/home/kaitlyn/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/kaitlyn/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/kaitlyn/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/kaitlyn/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to 

In [50]:
tester

nan

In [51]:
from sklearn.cluster import KMeans
#['Water (g)', 'Protein (g)', 'Fat (g)', 'Carbohydrate (g)', 'Total sugars (g)']
water = pd.Series(tester.Water, name='Water')
protein = pd.Series(tester.Protein, name='Protein')
fat = pd.Series(tester.Fat, name='Fat')
carbs = pd.Series(tester.Carbohydrate, name='Carbs')
sugars = pd.Series(tester['Total sugars'], name='Sugars')
X = pd.concat([water,protein,fat,carbs,sugars], axis=1)
X.fillna(0)
kmeans = KMeans(n_clusters=15, random_state=0)
kmeans.fit(X.dropna())

AttributeError: 'DataFrame' object has no attribute 'Water'

In [30]:
for key, value in tester['Water (g)'].items():
    if isinstance(value, str):
        if '(' in value:
            tester['Water (g)'][key] = value[1:-1]
        elif value=='Tr':
            tester['Water (g)'][key] = '0.0'
        elif value=='N':
            tester['Water (g)'][key] = '0.0'
        
tester['Water (g)'] = tester['Water (g)'].astype(float)